# Part I: Data Acquisition & Cleaning

In [ ]:
from typing import Literal
import pandas as pd
import numpy as np

# Load the dataset
def dataset_import(file_name:str, file_type: Literal["csv", "xlsx"]):
    if file_type not in {"csv", "xlsx"}:
        raise ValueError("File must be either csv or xlsx")
    df= pd.read_csv(f"./{file_name}.{file_type}")
    df.head()
    return df


def data_type_validation(df):
    """
    Validate and convert data types of the DataFrame columns, and resort the dataset by MatchDate
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    Returns:
    - pd.DataFrame: DataFrame with validated and converted data types.
    """
    df["MatchDate"] = pd.to_datetime(df["MatchDate"], errors="coerce")
    df.sort_values(by="MatchDate", inplace=True)

    # Make Season an ordered categorical
    df["Season"] = pd.Categorical(df["Season"], ordered=True)
    for column in ["HomeTeam", "AwayTeam", "FullTimeResult", "HalfTimeResult"]:
        df[column] = df[column].astype("category")
    
    return df

def data_standardisation(df):
    """
    Standardise textual data in the dataset:
    - Normalise team names and results
    """
    team_columns = ["HomeTeam", "AwayTeam"]
    for column in team_columns:
        df[column] = df[column].str.strip().str.title()

    result_columns = ["FullTimeResult", "HalfTimeResult"]
    for column in result_columns:
        df[column] = df[column].str.strip().str.upper()
    return df 


def duplicate_handling(df, keep: Literal["first", "last", False] ="first"):
    """
    Handle duplicate rows in the DataFrame based on specified subset of columns.
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - keep (str): Determines which duplicates to keep. Options are "first", "last", or False (drop all duplicates).
    Returns:
    - pd.DataFrame: DataFrame with duplicates handled.
    """
    df = df.drop_duplicates(keep=keep)
    return df 

def missing_values_handling(df, column, role: Literal["HomeTeam", "AwayTeam"], method: Literal["mean", "median"]):
    """
    Handle missing values in a specified column of the match dataset using specified methods.
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - column (str): The column in which to handle missing values.
    - role (Literal["HomeTeam", "AwayTeam"]): The role of the team for which to impute missing values.
    - method (Literal["mean", "median"]): The method to use for imputing missing values.
    Returns:
    - pd.DataFrame: DataFrame with missing values handled.
    """
    if role not in ["HomeTeam", "AwayTeam"]:
        raise ValueError("Role must be either 'HomeTeam' or 'AwayTeam'.")
    if method not in ["mean", "median"]:
        raise ValueError("Method must be either 'mean' or 'median'.")
    
    team_season_stats = (
        df.groupby(["Season", role])[column].agg(method).to_dict()
    )

    def fill_value(row):
        return team_season_stats.get((row["Season"], row[role]), np.nan)
    
    missing_mask = df[column].isnull()
    df.loc[missing_mask, column] = df[missing_mask].apply(fill_value, axis=1)

    return df

    

def outlier_handling(df, column, method: Literal["z_score", "iqr"], action: Literal["remove", "cap", "flag"], threshold: float = None):
    """
    Handle outliers in a specified column of the match dataset using specified methods.
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - column (str): The column in which to handle outliers.
    - method (Literal["z_score", "iqr"]): The method to use for detecting outliers.
    - action (Literal["remove", "cap", "flag"]): The action to take for handling outliers.
    - threshold (float, optional): The threshold value to use for outlier detection.    
    """
    if method not in ["z_score", "iqr"]:
        raise ValueError("Method must be either 'z_score' or 'iqr''.")
    if action not in ["remove", "cap", "flag"]:
        raise ValueError("Action must be either 'remove', 'cap', or 'flag'.")
    if column not in df.columns:
        raise ValueError(f"Column {column} does not exist in the DataFrame.")
    
    # Calculate outliers based on the specified method
    if method == "z_score":
        if threshold is None:
            threshold = 3
        upper = df[column].mean() + threshold * df[column].std()
        lower = df[column].mean() - threshold * df[column].std()
        mask = (df[column] > upper) | (df[column] < lower)
    elif method == "iqr":
        if threshold is None:
            threshold = 1.5
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        upper = q3 + threshold * iqr
        lower = q1 - threshold * iqr
        mask = (df[column] > upper) | (df[column] < lower)
    
    outliers_count = mask.sum()
    
    # Handle outliers based on the specified action
    if action == "remove":
        df = df[~mask]
    elif action == "cap":
        df[column] = np.where(df[column] > upper, upper, df[column])
        df[column] = np.where(df[column] < lower, lower, df[column])
    elif action == "flag":
        flag_column = f"Is_Outlier_{column}"
        df[flag_column] = mask
    return df

def logic_validation(df):
    """
    Check if the data are logical
    For example: the total number of cards a team receives in a match should be no more than the sum of fouls 
    """
    df = df[(df["HomeShotsOnTarget"] <= df["HomeShots"]) &
            (df["AwayShotsOnTarget"] <= df["AwayShots"])]
    
    df = df[(df["HalfTimeHomeGoals"] <= df["FullTimeHomeGoals"]) &
            (df["HalfTimeAwayGoals"] <= df["FullTimeAwayGoals"])]
    
    away_cards = df["AwayYellowCards"] + df["AwayRedCards"]
    home_cards = df["HomeYellowCards"] + df["HomeRedCards"]
    df = df[(away_cards <= df["AwayFouls"]) & (home_cards <= df["HomeFouls"])]

    # Full time result logic:
    mask_fulltime = (
        ((df["FullTimeResult"] == "H") & (df["FullTimeHomeGoals"] > df["FullTimeAwayGoals"])) |
        ((df["FullTimeResult"] == "A") & (df["FullTimeHomeGoals"] < df["FullTimeAwayGoals"])) |
        ((df["FullTimeResult"] == "D") & (df["FullTimeHomeGoals"] == df["FullTimeAwayGoals"]))
    )
    
    # Halftime result logic:
    mask_halftime = (
        ((df["HalfTimeResult"] == "H") & (df["HalfTimeHomeGoals"] > df["HalfTimeAwayGoals"])) |
        ((df["HalfTimeResult"] == "A") & (df["HalfTimeHomeGoals"] < df["HalfTimeAwayGoals"])) |
        ((df["HalfTimeResult"] == "D") & (df["HalfTimeHomeGoals"] == df["HalfTimeAwayGoals"]))
    )
    
    # Both conditions must be true
    df = df[mask_fulltime & mask_halftime]
    return df 

def numeric_validation(df, column):
    """
    Check for impossible numeric values like negatives.
    """
    df = df[df[column] >= 0]
    return df

def analytical_metrics(df):
# Goal difference between home and away teams    
    df["GoalDifference"] = df["FullTimeHomeGoals"] - df["FullTimeAwayGoals"]

# Total Cards per game
    df["TotalCards"] = df["HomeYellowCards"] + df["AwayYellowCards"] + df["HomeRedCards"] + df["AwayRedCards"]
    return df

df_raw = dataset_import("./Topic1_english_pl_dataset", "csv")
print(df_raw.describe())

df_raw =data_type_validation(df=df_raw)
df_raw = data_standardisation(df=df_raw)
df_preprocessed = duplicate_handling(df=df_raw, keep="first")
df_preprocessed = missing_values_handling(df=df_preprocessed, column="FullTimeAwayGoals", role="AwayTeam", method="median")
df_preprocessed = missing_values_handling(df=df_preprocessed, column="HalfTimeAwayGoals", role="AwayTeam", method="median")
df_preprocessed = missing_values_handling(df=df_preprocessed, column="HomeShotsOnTarget", role="HomeTeam", method="median")
df_preprocessed = missing_values_handling(df=df_preprocessed, column="HomeFouls", role="HomeTeam", method="median")
df_filled = missing_values_handling(df=df_preprocessed, column="AwayRedCards", role="AwayTeam", method="median")


# Select numeric columns except the excluded ones
numeric_columns = df_raw.drop(columns=["Season", "MatchDate", "HomeTeam", "AwayTeam", "FullTimeResult", "HalfTimeResult"]).select_dtypes(include=[np.number]).columns

for column in numeric_columns:
    df_cleaned = outlier_handling(df=df_filled, column=column, method="iqr", action="flag")

df_cleaned = logic_validation(df_cleaned)

for column in numeric_columns:
    df_cleaned = numeric_validation(df_cleaned, column)

df_processed = analytical_metrics(df_cleaned)
df_processed.to_csv("Processed_Dataset.csv", index=False)

# Part II: Visualisation

Overview over central functions of the following code paragraph:

calculate_match_points(row):
    This function calculates the points awarded for a single match based on the match result:
    - H (Home win): the home team receives three points
    - A (Away win): the away team receives three points
    - D (Draw): both teams receive one point each

create_season_table(df,season)
    This function creates the compplete statistics for a specific season:
    - Filters the data set according to the selected season
    - total points, fouls, yellow, and red cards for each team
    - returns the teams sorted by total points

**Objectives**
    - calculate points per team based on game results
    - consolidate relevant statistics such as fouls, yellow/red cards
    - visualize team performance (points) per season

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import colormaps
import seaborn as sns

df = pd.read_csv("Processed_Dataset.csv", parse_dates=["MatchDate"])
def calculate_match_points(row):
    """
    Calculates the points for the home and away teams of a match
    based on the FullTimeResult(H, A, D)
    """
    points = {
        row["HomeTeam"]: {"total": 0, "home": 0},
        row["AwayTeam"]: {"total": 0, "away": 0},
    }

    if row["FullTimeResult"] == "H":
        points[row["HomeTeam"]]["total"] = 3
        points[row["HomeTeam"]]["home"] = 3
    elif row["FullTimeResult"] == "A":
        points[row["AwayTeam"]]["total"] = 3
        points[row["AwayTeam"]]["away"] = 3
    elif row["FullTimeResult"] == "D":
        points[row["HomeTeam"]]["total"] = 1
        points[row["HomeTeam"]]["home"] = 1
        points[row["AwayTeam"]]["total"] = 1
        points[row["AwayTeam"]]["away"] = 1

    return points

# ---------------------------------------------------------------
# Creates an overview about the season statistics
# ---------------------------------------------------------------
def create_season_table(df, season: str):
    """
    creates a sorted season table of all the teams
    with corresponding points, fouls, yellow and red
    cards and other statistics for further analysis
    """
    df_season = df[df["Season"] == season]
    stats = {}

    for _, row in df_season.iterrows():
        # calculates the points
        points = calculate_match_points(row)

        # initializing the team
        for team in [row["HomeTeam"], row["AwayTeam"]]:
            if team not in stats:
                stats[team] = {
                    "Points": 0,
                    "FoulsCommitted": 0,
                    "YellowCards": 0,
                    "RedCards": 0,
                    "GamesPlayed": 0,
                    "HomePoints": 0,
                    "AwayPoints": 0,
                }

        # adding points to fill the respective dictionary spot
        for team, pts in points.items():
            stats[team]["Points"] += pts["total"]
            stats[team]["GamesPlayed"] += 1
            stats[team]["HomePoints"] += pts.get("home", 0)
            stats[team]["AwayPoints"] += pts.get("away", 0)

        # fill in the values for red and yellow cards
        stats[row["HomeTeam"]]["FoulsCommitted"] += row["HomeFouls"]
        stats[row["AwayTeam"]]["FoulsCommitted"] += row["AwayFouls"]
        stats[row["HomeTeam"]]["YellowCards"] += row["HomeYellowCards"]
        stats[row["AwayTeam"]]["YellowCards"] += row["AwayYellowCards"]
        stats[row["HomeTeam"]]["RedCards"] += row["HomeRedCards"]
        stats[row["AwayTeam"]]["RedCards"] += row["AwayRedCards"]

    # sort the list based on the number of points each team gained
    sorted_stats = dict(sorted(stats.items(), key=lambda x: x[1]["Points"], reverse=True))
    return sorted_stats

# ---------------------------------------------------------------
# Visualization of the results of the respective season
# ---------------------------------------------------------------
def plot_season_points(sorted_stats: dict, season: str):
    """
    creates a bar chart of the points for a season
    """
    teams = list(sorted_stats.keys())
    points = [team_stats["Points"] for team_stats in sorted_stats.values()]

    norm = Normalize(vmin=min(points), vmax=max(points))
    colormap = colormaps['Greys_r']
    colors = [colormap(norm(p)) for p in points]

    fig, ax = plt.subplots(figsize=(20, 5))
    ax.bar(teams, points, color=colors, edgecolor='black', linewidth=0.7, width=0.6, alpha=0.9)

    ax.set_xlabel("Clubs", fontsize=14, fontweight="bold")
    ax.set_ylabel("Points", fontsize=14, fontweight="bold")
    ax.set_title(f"Premier League: Season {season}", fontsize=18, fontweight="bold")
    plt.xticks(rotation=90)
    plt.show()
"""
This section displays the progression line of the champion of a selected season over all available season (1st place in the table).
The system then checks how this club has performed in the other seasons.


**Objectives**:
    - Visualization of the season rankings of the selected champion
    - Insights into long-term performance (e.g. does the champion remain strong or not?)
"""                                           
# ---------------------------------------------------------------
# Funktion: calculating the champion's placement over all seasons
# ---------------------------------------------------------------
def champion_placement_over_time(df, input_season: str):
    """
    Calculates and visualizes the development of the champion's placement
    in a chosen season over all seasons, shown in the data set.
    """
    # determining the champion in the chosen season
    first_table = create_season_table(df, input_season)
    targeted_team = list(first_table.keys())[0]  # Champion = first key

    # Sorting all available seasons
    all_seasons_list = sorted(df['Season'].unique())

    placement_bestTeam = {}  # Dictionary: Season -> placement: Champion

    # Iterate across all seasons and search the champion's placement in that specific season
    for season in all_seasons_list:
        stats = create_season_table(df, season)   # FIXED: was tcreate_season_table
        clubs = list(stats.keys())            # list of the listed teams in that season

        if targeted_team in clubs:
            # Index +1, since placement zero isn't existing
            curr_season_placement = clubs.index(targeted_team) + 1
        else:
            # If the team is not participating in the respective season
            curr_season_placement = None

        placement_bestTeam[season] = curr_season_placement

    # Visualization
    seasons = list(placement_bestTeam.keys())
    positions = list(placement_bestTeam.values())

    plt.figure(figsize=(12, 6))
    plt.plot(seasons, positions, marker='o', color='crimson', linewidth=2)
    plt.gca().invert_yaxis()
    plt.xticks(rotation=45)
    plt.title(f"Overall Placement Line of {targeted_team}", fontsize=16, fontweight='bold')
    plt.xlabel("Season", fontsize=14, fontweight='bold')
    plt.ylabel(f"Placement (Champion: {targeted_team}) in Season {input_season}",
               fontsize=14, fontweight='bold')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ---------------------------------------------------------------
# Function: displaying home/away performances and differences
# ---------------------------------------------------------------
def plot_home_away_performance(stats: dict, input_season: str):
    """
    Visualizes Home and Away performances and their differences for all teams
    in a chosen season.

    Parameter:
    ----------
    stats : dict
        Dictionary of the seasonal statistics (create_season_table).
    input_season : str
        Season which is to be shown (z. B. "2000/01").
    """

    # -------------------------------------------
    # DataFrames for HomePoints and AwayPoints
    # -------------------------------------------
    df_home = pd.DataFrame.from_dict(
        {team: points["HomePoints"] for team, points in stats.items()},
        orient="index", columns=["HomePoints"]
    ).sort_values(by="HomePoints", ascending=False)

    df_away = pd.DataFrame.from_dict(
        {team: points["AwayPoints"] for team, points in stats.items()},
        orient="index", columns=["AwayPoints"]
    ).sort_values(by="AwayPoints", ascending=False)

    # -------------------------------------------
    # DataFrame for difference (Home - Away)
    # -------------------------------------------
    df_diff = pd.DataFrame.from_dict(
        {team: points["HomePoints"] - points["AwayPoints"] for team, points in stats.items()},
        orient="index", columns=["Point Difference"]
    ).sort_values(by="Point Difference", ascending=False)

    # -------------------------------------------
    # Plot: 3 locating the heat maps next to each other
    # -------------------------------------------
    fig, axs = plt.subplots(1, 3, figsize=(18, len(df_home) * 0.4))

    # Heatmap 1: home-performance
    sns.heatmap(df_home, annot=True, cmap="Reds", linewidths=0.5, ax=axs[0])
    axs[0].set_title("Strongest Home Performance")
    axs[0].set_xlabel("Points")
    axs[0].set_ylabel(f"Teams (Season: {input_season})", fontweight='bold')

    # Heatmap 2: away-performance
    sns.heatmap(df_away, annot=True, cmap="Blues", linewidths=0.5, ax=axs[1])
    axs[1].set_title("Strongest Away Performance")
    axs[1].set_xlabel("Points")
    axs[1].set_ylabel("")

    # Heatmap 3: difference home - away
    sns.heatmap(df_diff, annot=True, cmap="coolwarm", center=0, linewidths=0.5, ax=axs[2])
    axs[2].set_title("Biggest Difference between Home/Away Performance")
    axs[2].set_xlabel("Diff")
    axs[2].set_ylabel("")

    plt.tight_layout()
    plt.show()
"""
Participation matrix: Teams per season

This section shows which teams were represented in which Premier League seasons. Each field shows 1 if a team played in the respective season, otherwise 0.
This makes it easy to see:
    - which teams have participated continuously (e.g. big clubs)
    - which teams have been promoted and relegated
    - in which seasons new teams have joined

The teams are sorted by the number of times they have participated  (most played at the top).
"""
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------------------
# Function: Creating and visualizing Participating-Matrix
# ---------------------------------------------------------------
def plot_team_participation_matrix(df: pd.DataFrame):
    """
    Creates a heatmap showing in which seasons which specific teams
    participated in the Premier League

    Parameter:
    ----------
    df : pd.DataFrame
        complete Premier-League-Data-Set with columns "HomeTeam", "AwayTeam" and "Season".
    """

    # -----------------------------------------------------------
    # determining all seasons and Teams
    # -----------------------------------------------------------
    seasons = sorted(df["Season"].unique())
    teams = sorted(set(df["HomeTeam"]).union(set(df["AwayTeam"])))

    print(f"Total number of different Teams which participated in the EPL from 2000-2025: {len(teams)}")

    # -----------------------------------------------------------
    # initializing empty Matrix (Teams as index, seasons as columns)
    # -----------------------------------------------------------
    presence_matrix = pd.DataFrame(0, index=teams, columns=seasons)

    # -----------------------------------------------------------
    # filling the matrix: 1, if the team partcicpated in the respective season
    # -----------------------------------------------------------
    for season in seasons:
        season_df = df[df["Season"] == season]
        teams_in_season = set(season_df["HomeTeam"]).union(set(season_df["AwayTeam"]))

        for team in teams_in_season:
            presence_matrix.loc[team, season] = 1

    # -----------------------------------------------------------
    # Sorting teams based on their number of participations in the Premier League (first league!)
    # -----------------------------------------------------------
    participation = presence_matrix.sum(axis=1)
    presence_matrix_sorted = presence_matrix.loc[participation.sort_values(ascending=False).index]

    # -----------------------------------------------------------
    # creating the heatmap
    # -----------------------------------------------------------
    plt.figure(figsize=(11, 11))
    sns.heatmap(presence_matrix_sorted, cmap="Greens", cbar=False, linewidths=0.2)
    plt.title("Participations of Teams per Season (1 = participated)", fontweight='bold')
    plt.xlabel("Season", fontweight='bold')
    plt.ylabel("Teams", fontweight='bold')
    plt.tight_layout()
    plt.show()

# Part III: Modeling

Defines a machine learning pipeline to predict a football team's match outcomes for a specific season using only that team's historical match data from previous seasons. It preprocesses match statistics and team names, trains a Random Forest classifier, and evaluates the predictions with standard classification metrics.

In [ ]:
# Soccer Match Prediction Pipeline

# Import all the tools we'll need
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, accuracy_score,
    f1_score, balanced_accuracy_score,
    confusion_matrix
)


# Define cleaned DataFrame
df_clean = pd.read_csv("./Processed_Dataset.csv", parse_dates=["MatchDate"])



# Team-Specific, Season-Aware Prediction Pipeline


def predict_team_matches(df, team_name: str, predict_season: int):
    """
    Predicts the match results for a specific team for a given season
    using only historical data before that season.
    
    Parameters:
    - df (pd.DataFrame): Cleaned dataset
    - team_name (str): Name of the team to predict for
    - predict_season (int): Season year to predict (e.g. 2023 means 2023-24)
    
    Returns:
    - pd.DataFrame with actual and predicted results
    """
    # Filter team matches (home or away)
    team_df = df[(df["HomeTeam"] == team_name) | (df["AwayTeam"] == team_name)].copy()

    # Keep only the seasons needed
    train_df = team_df[team_df["MatchDate"].dt.year < predict_season]
    test_df = team_df[team_df["MatchDate"].dt.year == predict_season]

    if train_df.empty or test_df.empty:
        print(f"No data available for training or testing for team '{team_name}' in season {predict_season}.")
        return None

    # Define predictors and label
    match_stats = [
        'HalfTimeHomeGoals', 'HalfTimeAwayGoals',
        'HomeShots', 'AwayShots',
        'HomeShotsOnTarget', 'AwayShotsOnTarget',
        'HomeCorners', 'AwayCorners',
        'HomeFouls', 'AwayFouls',
        'HomeYellowCards', 'AwayYellowCards',
        'HomeRedCards', 'AwayRedCards'
    ]
    team_identifiers = ['HomeTeam', 'AwayTeam']
    predictors = match_stats + team_identifiers

    # Features and labels
    X_train = train_df[predictors]
    y_train = train_df["FullTimeResult"]
    X_test = test_df[predictors]
    y_test = test_df["FullTimeResult"]

    # Encode results
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Preprocessing
    numeric_prep = Pipeline([('scaler', StandardScaler())])
    team_prep = Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer([
        ('stats', numeric_prep, match_stats),
        ('teams', team_prep, team_identifiers)
    ])

    # Final model pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
    ])

    # Train and predict
    pipeline.fit(X_train, y_train_encoded)
    predictions = pipeline.predict(X_test)

    # Evaluation
    print(f"\nPredictions for {team_name} in Season {predict_season}:")
    print("--------------------------------------------------------")
    print(f"Accuracy: {accuracy_score(y_test_encoded, predictions):.2%}")
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test_encoded, predictions):.2%}")
    print(f"F1 Score: {f1_score(y_test_encoded, predictions, average='macro'):.2%}")
    print("\nClassification Report:")
    print(classification_report(y_test_encoded, predictions, target_names=label_encoder.classes_))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_encoded, predictions))

    # Return a DataFrame with actual vs predicted
    test_df = test_df.copy()
    test_df["PredictedResult"] = label_encoder.inverse_transform(predictions)
    return test_df[["MatchDate", "HomeTeam", "AwayTeam", "FullTimeResult", "PredictedResult"]]


# Example Usage
"""
-Enter team name for the outcome you want to predict
-Enter season you want to predict
-Uses previous seasons of the specific team as data-set and outputs the season's prediction
 
"""

team_name = "Man United"
season_to_predict = 2023

results_df = predict_team_matches(df_clean, team_name, season_to_predict)
if results_df is not None:
    print(results_df.head())

